In [1]:
import pandas as pd
import dash as ds
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
import plotly.express as px

# 1. height vs position
# 2. avg value of players from different countries
# 3. leagues comparison(value)



In [2]:
data = pd.read_csv("transfermarkt_fbref_201920.csv", sep=";")
data.head()

,Column1,player,nationality,position,squad,age,birth_year,value,height,position2,...,Pts/G,xG,xGA,xGDiff,xGDiff/90,Attendance,CL,WinCL,CLBestScorer,Season
0,21,Martin Aguirregabiria,es ESP,DF,AlavĂ©s,23,1996,4000000,178,Defender - Right-Back,...,1.03,37.0,53.2,-16.2,-0.43,"12,177",0,0,0.0,201920#
1,390,Oliver Burke,sco SCO,"MF,FW",AlavĂ©s,22,1997,4000000,188,Forward - Right Winger,...,1.03,37.0,53.2,-16.2,-0.43,"12,177",0,0,0.0,201920#
2,430,VĂ­ctor Camarasa,es ESP,MF,AlavĂ©s,25,1994,4000000,183,Midfielder - Central Midfield,...,1.03,37.0,53.2,-16.2,-0.43,"12,177",0,0,0.0,201920#
3,737,RubĂ©n Duarte,es ESP,DF,AlavĂ©s,23,1995,4000000,172,Defender - Left-Back,...,1.03,37.0,53.2,-16.2,-0.43,"12,177",0,0,0.0,201920#
4,770,Rodrigo Ely,br BRA,DF,AlavĂ©s,25,1993,1000000,188,Defender - Centre-Back,...,1.03,37.0,53.2,-16.2,-0.43,"12,177",0,0,0.0,201920#


In [3]:
player_counts_age_league = data.groupby(['age', 'league']).size().reset_index(name='count')
player_counts_age_league = player_counts_age_league.loc[player_counts_age_league['age'] != 0]

print(player_counts_age_league)

data["nationality"] = data["nationality"].str.slice(3,6)

data["nationality"] = data["nationality"].replace(' EN', 'GBR')
data["nationality"] = data["nationality"].replace(' WA', 'GBR')
data["nationality"] = data["nationality"].replace(' SC', 'GBR')
data["nationality"] = data["nationality"].replace('GER', 'DEU')
data["nationality"] = data["nationality"].replace('POR', 'PRT')
data["nationality"] = data["nationality"].replace('DEN', 'DNK')
data["nationality"] = data["nationality"].replace('NED', 'NLD')
data["nationality"] = data["nationality"].replace('SUI', 'CHE')
data["nationality"] = data["nationality"].replace('URU', 'URY')
data["nationality"] = data["nationality"].replace('PAR', 'PRY')
data["nationality"] = data["nationality"].replace('RSA', 'ZAF')
data["nationality"] = data["nationality"].replace('CRO', 'HRV')
data["nationality"] = data["nationality"].replace('GRE', 'GRC')
data["nationality"] = data["nationality"].replace('CHI', 'RCH')
data["nationality"] = data["nationality"].replace('GAM', 'GMB')
data["nationality"] = data["nationality"].replace('ALG', 'DZA')

player_counts = data['nationality'].value_counts().reset_index()
player_counts.columns = ['country', 'count']


pd.set_option('display.max_rows', None)



     age      league  count
2     14     La Liga      1
3     15     Ligue 1      1
4     16  Bundesliga      3
5     16     La Liga      1
6     16     Ligue 1      2
..   ...         ...    ...
111   38     Serie A      1
112   39     Ligue 1      1
113   40  Bundesliga      1
114   41     Ligue 1      1
115   41     Serie A      1

[114 rows x 3 columns]


In [4]:
#Sum of 11 most valued players per country
data["nationality"].nunique()
print(data["nationality"].isna().sum())
print(data["value"].isna().sum())

avg_value_per_country = data.groupby("nationality")["value"].nlargest(11)

sum_top_10_players_per_country = avg_value_per_country.groupby('nationality').sum()
sorted_players_sum = sum_top_10_players_per_country.sort_values(ascending=False).head(20)

sorted_players_sum_df = sorted_players_sum.to_frame('sum_value')
sorted_players_sum_df['continent'] = ['Europe', 'Europe', 'South America', 'Europe', 'Europe',
                                       'South America', 'Europe', 'Europe', 'Europe', 'Europe',
                                         'South America', 'Europe', 'Africa', 'Africa', 'Europe', 
                                         'Europe', 'Europe','South America', 'Asia', 'Europe']

0
0


In [86]:
#clean positions
data["position"] = data["position"].str.slice(0, 2)
data["position"].isna().sum()
data["position"].nunique()
data_for_height = data[data["height"] > 140]
data= data[data["goals"] > 1]

In [80]:
data = pd.read_csv("transfermarkt_fbref_201920.csv", sep=";")
data['Attendance'] = data['Attendance'].str.replace(',', '').astype(int)

team_values = data.groupby('squad')['value'].sum().reset_index()
team_attendance = data.groupby('squad')['Attendance'].first().reset_index()

goals_per_team_league = data.groupby(['squad', 'league'])['goals'].sum().reset_index()
goals_per_team_league_attendance = pd.merge(goals_per_team_league, team_attendance, on = 'squad', how= 'left')
teams_per_value_per_goals_per_league = pd.merge(goals_per_team_league_attendance, team_values, on='squad', how='left')
teams_per_value_per_goals_per_league['value'] = teams_per_value_per_goals_per_league['value'] / 1e6
teams_per_value_per_goals_per_league = teams_per_value_per_goals_per_league.rename(columns={'value': 'value_in_millions'})

In [89]:
app = ds.Dash(__name__)

# Create choropleth map
color_scale = ["#b3fcb3", "#63c402", "#32CD32", "#FFFF00", "#FFD700", "#FFA500", "#FF8000", "#FF4500","#ff5454", "#FF0000", "red", "red" ]
fig1 = px.choropleth(player_counts, locations='country', color='count',
                    locationmode='ISO-3',
                    color_continuous_scale= color_scale,
                    title='Counts of Players per Nationality in Each Country')
fig1.update_layout(height=800, width=1400)


fig2 = px.bar(sorted_players_sum_df, x=sorted_players_sum.index, y='sum_value', color = 'continent', text_auto='.2s',
              title = "Sum of values of 11 most valued players per country 2020",
              labels= {"sum_value": "Total Value (milions)", "nationality":"Country Code"})
fig2.update_layout(font_color = 'black', title_font_family = 'Arial Black', font=dict(size=14))
fig2.update_xaxes(categoryorder='total descending', title_font_family = 'Arial')
fig2.update_yaxes(tickfont_family="Arial Black")


fig3 = px.line(player_counts_age_league, x='age', y='count', color='league', title='Number of Players per Each Age in Each League')
fig3.update_layout(font_color = 'black', title_font_family = 'Arial Black', font=dict(size=14))


fig4 = px.scatter(teams_per_value_per_goals_per_league , x='Attendance', y='goals', size='value_in_millions', color='league',
                 title='Team goals scored vs Attendance per Value per League', 
                 labels={'squad':'Team', 'league':'League', 'team_value':'Team Value (Milions)', 'goal':'Number of Goals'}, hover_data=['squad'])
fig4.update_layout(font_color = 'black', title_font_family = 'Arial Black',font=dict(size=14))
fig4.update_yaxes(tickfont_family="Arial Black")
fig4.update_xaxes(tickfont_family="Arial Black")

fig5 = px.scatter(data, x = "minutes", y = 'goals', color = "position", title='Player Goals scored vs minutes played per position.',hover_data=['player','squad'])
fig5.update_layout(font_color = 'black', title_font_family = 'Arial Black',font=dict(size=14))
fig5.update_yaxes(tickfont_family="Arial Black")
fig5.update_xaxes(tickfont_family="Arial Black")

app.layout = html.Div(children=[
    html.H1(children='Lab Part Dashboard'),
    dcc.Graph(
        id='graph1',
        figure=fig1
    ),
    dcc.Graph(
        id='graph2',
        figure=fig2
    ),
    dcc.Graph(
        id='graph3',
        figure=fig3
    ),
    dcc.Graph(
        id='graph4',
        figure=fig4
    ),
    dcc.Graph(
        id='graph5',
        figure=fig5
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8081)